### Download HRRR data from U of UTAH CHPC archive
Wei-Ming Tsai, Nov. 2020 <br>
HRRR data archive: http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/hrrr_script_tips.html

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cfgrib
import cf2cdm
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import xarray as xr
from datetime import datetime, timedelta
import urllib.request
from cfgrib.xarray_store import open_dataset
import warnings
import h5py

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# var = 'PWAT'
# id_begin = 49173818
# id_end = 50061135

In [4]:
os.chdir('/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/pic/')
file = np.loadtxt('rms_timeseriesResidual_ramp.txt',skiprows=4)
date_acqui = []
for t in range(file.shape[0]):
    date_acqui.append(str(file[t,0])[:8]) # times of acquisitions

In [5]:
date_acqui

['20150322',
 '20150403',
 '20150415',
 '20150509',
 '20150626',
 '20150720',
 '20150906',
 '20160104',
 '20160221',
 '20160316',
 '20160409',
 '20160503',
 '20160527',
 '20160714',
 '20160807',
 '20161105',
 '20161129',
 '20161223',
 '20170116',
 '20170305',
 '20170317',
 '20170422',
 '20170516',
 '20170528',
 '20170609',
 '20170621',
 '20170703',
 '20170715',
 '20170727',
 '20170808',
 '20170820',
 '20170901',
 '20170913',
 '20170925',
 '20171007',
 '20171019',
 '20171031',
 '20171112',
 '20171124',
 '20171206',
 '20171218',
 '20171230',
 '20180111',
 '20180123',
 '20180204',
 '20180216',
 '20180228',
 '20180312',
 '20180324',
 '20180405',
 '20180417',
 '20180429',
 '20180511',
 '20180523',
 '20180604',
 '20180616',
 '20180710',
 '20180722',
 '20180803',
 '20180815',
 '20180908',
 '20180920',
 '20181002',
 '20181014',
 '20181026',
 '20181107',
 '20181119',
 '20181201',
 '20181213',
 '20181225',
 '20190106',
 '20190118',
 '20190130',
 '20190211',
 '20190223',
 '20190319',
 '20190331',

In [ ]:
len(date_acqui)

### Download HRRR data from archive

In [6]:
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_data/google_archive/')
# Downloading best analyzed output for specfied UTC.
# example: 
for date in date_acqui:
    date_str = date
    hour_str = '02'
    file_name = 'hrrr.'+date_str+'.t'+hour_str+'z.grib2'
    url = 'curl -o ' + file_name + ' '\
      'https://storage.googleapis.com/high-resolution-rapid-refresh/hrrr.'+date_str+'/conus/hrrr.t'+hour_str+'z.wrfprsf00.grib2'
    os.system(url)
    print(file_name)

hrrr.20150322.t02z.grib2
hrrr.20150403.t02z.grib2
hrrr.20150415.t02z.grib2
hrrr.20150509.t02z.grib2
hrrr.20150626.t02z.grib2
hrrr.20150720.t02z.grib2
hrrr.20150906.t02z.grib2
hrrr.20160104.t02z.grib2
hrrr.20160221.t02z.grib2
hrrr.20160316.t02z.grib2
hrrr.20160409.t02z.grib2
hrrr.20160503.t02z.grib2
hrrr.20160527.t02z.grib2
hrrr.20160714.t02z.grib2
hrrr.20160807.t02z.grib2
hrrr.20161105.t02z.grib2
hrrr.20161129.t02z.grib2
hrrr.20161223.t02z.grib2
hrrr.20170116.t02z.grib2
hrrr.20170305.t02z.grib2
hrrr.20170317.t02z.grib2
hrrr.20170422.t02z.grib2
hrrr.20170516.t02z.grib2
hrrr.20170528.t02z.grib2
hrrr.20170609.t02z.grib2
hrrr.20170621.t02z.grib2
hrrr.20170703.t02z.grib2
hrrr.20170715.t02z.grib2
hrrr.20170727.t02z.grib2
hrrr.20170808.t02z.grib2
hrrr.20170820.t02z.grib2
hrrr.20170901.t02z.grib2
hrrr.20170913.t02z.grib2
hrrr.20170925.t02z.grib2
hrrr.20171007.t02z.grib2
hrrr.20171019.t02z.grib2
hrrr.20171031.t02z.grib2
hrrr.20171112.t02z.grib2
hrrr.20171124.t02z.grib2
hrrr.20171206.t02z.grib2


### Extract precipitable water and merge into a single ncfile

In [ ]:
# get geolocation from InSAR
geo_file = '/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/inputs/geometryRadar.h5'
geo = h5py.File(geo_file,'r')
# for key in geo.keys():
#     print(key) #Names of the groups in HDF5 file.
lat = geo['latitude'];
lon = geo['longitude'];
incidence = geo['incidenceAngle'];
axis_bound = [np.min(lat),np.max(lat),np.min(lon),np.max(lon)]; # coordinate bound [South,North,West,East]
axis_bound = [np.unique(lat.value)[1],np.unique(lat.value)[-1],np.unique(lon.value)[0],np.unique(lon.value)[-2]]
axis_bound